# Model Training
Train the model from the /train folder. Gets class types through the names of folders each image is in. Then saves the model to the project directory.

In [ ]:
import numpy as np
import tensorflow as tf
import cv2

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping
from keras_visualizer import visualizer 

In [ ]:
import pathlib

DATA_DIR = pathlib.Path("./train")
IMAGE_COUNT = len(list(DATA_DIR.glob('*/*.jpg')))
print(IMAGE_COUNT)

In [ ]:
BATCH_SIZE = 64
IMG_HEIGHT = 48
IMG_WIDTH = 48

INPUT_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 1)

train_ds = tf.keras.utils.image_dataset_from_directory(
  DATA_DIR,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE,
  color_mode="grayscale")

val_ds = tf.keras.utils.image_dataset_from_directory(
  DATA_DIR,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE,
  color_mode="grayscale")

CLASS_NAMES = train_ds.class_names
NUM_CLASSES = len(CLASS_NAMES)

print(CLASS_NAMES)

### Example Data Image + Labels

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"), cmap="gray")
    plt.title(CLASS_NAMES[labels[i]])
    plt.axis("off")

### Model Creation

In [ ]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
WEIGHT_DECAY = 0.0008

model = Sequential([
  # the following three data augmentation layers are not allowed in TFJS
  # layers.Resizing(img_height, img_width, input_shape=(img_height, img_width, 3)),
  # layers.Rescaling(1./255),
  # layers.RandomRotation(0.2),
  layers.Conv2D(
    32,
    kernel_size = (3, 3),
    activation = "relu",
    input_shape = INPUT_SHAPE,
    kernel_regularizer=tf.keras.regularizers.L1L2(l1=0, l2=WEIGHT_DECAY),
    bias_regularizer=tf.keras.regularizers.L1L2(l1=0, l2=WEIGHT_DECAY)
  ),
  layers.Conv2D(
    64,
    (3, 3),
    activation = "relu",
    kernel_regularizer=tf.keras.regularizers.L1L2(l1=0, l2=WEIGHT_DECAY),
    bias_regularizer=tf.keras.regularizers.L1L2(l1=0, l2=WEIGHT_DECAY)
  ),
  layers.Conv2D(
    128,
    (4, 4), 
    activation = "relu", 
    kernel_regularizer=tf.keras.regularizers.L1L2(l1=0, l2=WEIGHT_DECAY),
    bias_regularizer=tf.keras.regularizers.L1L2(l1=0, l2=WEIGHT_DECAY)
  ),
  layers.MaxPooling2D(pool_size = (2, 2)),
  layers.Dropout(0.5),
  layers.Flatten(),
  layers.Dense(
    128, 
    activation = 'relu', 
    kernel_regularizer=tf.keras.regularizers.L1L2(l1=0, l2=WEIGHT_DECAY),
    bias_regularizer=tf.keras.regularizers.L1L2(l1=0, l2=WEIGHT_DECAY)
  ),
  layers.Dropout(0.5),
  layers.Dense(NUM_CLASSES, activation = "softmax"),
])

model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

model.summary()
# visualizer(model, format='png', view=True) # visualize the model as an image

### Train

In [ ]:
EPOCHS = 100
model_folder = "./model/"
model_weights_best = model_folder + "roggle_letter_classifier.weights.best.hdf5"


# Save weights only for best model
checkpointer = ModelCheckpoint(filepath = model_weights_best, 
                               verbose = 2, 
                               save_best_only = True)

# Reduce learning rate when a metric has stopped improving
lr_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                 patience=4, 
                                 verbose=2, 
                                 factor=.75)

# If score doesn't improve, stop learning
estopping = EarlyStopping(monitor='val_loss', 
                          patience=5, 
                          verbose=2)

history = model.fit(
  train_ds,
  batch_size = BATCH_SIZE,
  verbose = 1,
  validation_data=val_ds,
  epochs=EPOCHS,
  callbacks = [checkpointer, lr_reduction, estopping]
)

In [ ]:
# save the final model
model_name = "roggle_letter_classifier"
model_path = model_folder + model_name
model.save_weights(model_path)
model.save(model_path, save_format='h5')

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(estopping.stopped_epoch)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

### Inference

In [ ]:
model = tf.keras.models.load_model(model_weights_best)

test_images = pathlib.Path("./test").glob('*.jpg')

for img_path in test_images:
    img_path = str(img_path).replace('\\', '/')
    print(img_path)
    img = tf.keras.utils.load_img(
        img_path, 
        target_size=(IMG_HEIGHT, IMG_HEIGHT), 
        color_mode="grayscale"
    )

    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])


    image_to_display = cv2.imread(img_path)
    image_to_display = cv2.cvtColor(image_to_display, cv2.COLOR_BGR2RGB)

    print()
    plt.imshow(image_to_display)
    plt.title("[{}]:\n{} - {:.2f}".format(img_path, CLASS_NAMES[np.argmax(score)],  10 * np.max(score)))
    plt.show()

### Convert from Keras model format to TFJS format


In [ ]:
import tensorflowjs as tfjs

tfjs.converters.save_keras_model(model, "./model_tfjs")